<a href="https://colab.research.google.com/github/shruthireddyrekula/Binary_Classification_Diabetes_Dataset_Pytorch/blob/main/Diabetes_prediction_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
# load the dataset, split into input (X) and output (y) variables
dataset = np.loadtxt('/content/Datasets_master_pima-indians-diabetes_data.csv', delimiter=',')
X = dataset[:,0:8]
y = dataset[:,8]

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
class PimaClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(8, 12)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(12, 8)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(8, 1)
        self.act_output = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act_output(self.output(x))
        return x

model = PimaClassifier()
print(model)

In [17]:
loss_fn = nn.BCELoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:

n_epochs = 100
batch_size = 10

for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size]
        y_pred = model(Xbatch)
        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')

In [19]:
# compute accuracy (no_grad is optional)
with torch.no_grad():
    y_pred = model(X)

accuracy = (y_pred.round() == y).float().mean()
print(f"Accuracy {accuracy}")

Accuracy 0.7654722929000854


In [23]:
# make probability predictions with the model
predictions = model(X)
# round predictions
rounded = predictions.round()

In [24]:
# make class predictions with the model
predictions = (model(X) > 0.5).int()

In [ ]:
model.eval()

# Initialize lists to store predictions and true labels
all_preds = []
all_labels = []

# Compute predictions
with torch.no_grad():
    outputs = model(X_test)
    predicted = torch.round(outputs)  # Threshold the predictions at 0.5
    all_preds.extend(predicted.numpy())
    all_labels.extend(y_test)

# Calculate accuracy
correct = (predicted == y_test).float().sum()
accuracy = correct / len(y_test)

print(f'Accuracy on test data: {accuracy:.4f}')